# Prep

In [104]:
import pandas as pd
import numpy as np
import pprint
import re
import os
import psycopg2
import janitor

from dotenv import load_dotenv
from psycopg2 import sql, extras

In [105]:
WORKOUTS_FILE = r'C:\Users\User\Documents\GitHub\ap_workouts\b1-bronze\workouts.csv'
PLANS_FILE = r'C:\Users\User\Documents\GitHub\ap_workouts\b1-bronze\plans.csv'
CUSTOMEXERCISES_FILE = r'C:\Users\User\Documents\GitHub\ap_workouts\b1-bronze\custom_exercises.csv'
SILVER_FOLDER = r'C:\Users\User\Documents\GitHub\ap_workouts\b2-silver'
GOLD_FOLDER = r'C:\Users\User\Documents\GitHub\ap_workouts\b3-gold'

## Auth

In [106]:
load_dotenv(r"C:\\Users\\User\\Documents\\GitHub\\notion_api\\postgres_auth.env")

db_name = os.getenv("dbname")
db_user = os.getenv("user")
db_password = os.getenv("password")
db_host = os.getenv("host")
db_port = os.getenv("port")

## Functions

### Extract

In [107]:
def get_raw_workouts(fp):
  all_workouts = []
  counter = 0

  with open(fp, encoding='utf-8') as f:
    clean_strings = [line.strip() for line in f.readlines()]
    no_empty_lines = list(filter(None, clean_strings))
    no_empty_lines.pop(0) # tirar a primeira linha, que é vazia

    single_workout = []
    for line in no_empty_lines:
      if 'Week' in line or 'Standalone' in line:
        if counter > 0:
          all_workouts.append(single_workout)
          single_workout = []
        counter += 1
      single_workout.append(line)

  return all_workouts

### Transform

#### Flatten - Big Function

In [108]:
def get_workout_dict(raw_workout_list):
    workout_listdict = []

    workout_counter = 0
    for workout in raw_workout_list:
        #print(f"Workout: {workout_counter}") # use to find errors

        workout_description = workout[0].split(';')
        description_info = [item.strip() for item in workout_description[0].strip('"').split('·')]

        # 
        workout_dict = {
            #'workout_counter': workout_counter, # use to find errors
            #'raw_description': f'{workout[0]}', # use to check for data quality
            'description': {
                'id': re.sub(r'\D', '', workout_description[1].strip('"')), #re.sub(r'\D', '', input_string) ## workout_description[1].strip('"').strip('h').strip(' ')
                'day_name': description_info[0],
                'day_number': description_info[1],
                'week_number': '',
                'plan_name': '',
                'plan_function': '',
                'start_time': workout_description[1].strip('"'),
                'duration': workout_description[2].strip('"')
            },
            #'raw_exercises': [],
            'exercises': []
        }

        # Standalone workouts follow a different structure and need specific treatment
        if 'Standalone' in workout[0]:
            workout_dict['description']['week_number'] = '0'
            workout_dict['description']['plan_name'] = 'Standalone'
            try:
                workout_dict['description']['plan_function'] = description_info[2]
            except:
                workout_dict['description']['plan_function'] = ''
        else:
            workout_dict['description']['week_number'] = description_info[2]
            workout_dict['description']['plan_name'] = description_info[3]
            try:
                workout_dict['description']['plan_function'] = description_info[4]
            except:
                workout_dict['description']['plan_function'] = ''

        # 
        raw_current_exercise = {
            'exercise': '',
            'data': []
        }
        raw_exercises = []
        for line in workout[1:]:
            if len(line) > 13:
                if len(raw_current_exercise['exercise']) > 0:
                    raw_exercises.append(raw_current_exercise)
                raw_current_exercise = {'exercise': line, 'data': []}
            else:
                raw_current_exercise['data'].append(line)
        raw_exercises.append(raw_current_exercise) # necessary to append the last dict

        # 
        clean_exercises = []
        for e in raw_exercises:
            exercise_sets = []
            #print(f"workout: {workout_counter}  {e}") # use to find errors

            raw_exercise_header = e['exercise']
            if ';' in e['exercise']:
                split_header = raw_exercise_header.split(';')
                exercise_info = [item.strip() for item in split_header[0].strip('"').split('·')]
                exercise_comments = split_header[1]
            else:
                exercise_info = [item.strip() for item in raw_exercise_header.strip('"').split('·')]
                exercise_comments = ''
            order_and_name = exercise_info[0].split(". ")

            clean_exercise = {
                #'info': exercise_info, # use to check data quality
                'workout_id': workout_dict['description']['id'],
                'exercise_number': order_and_name[0],
                'exercise_name': order_and_name[1],
                'equipment': exercise_info[1],
                'goal': np.nan
            }

            if (len(exercise_info) > 2):
                clean_exercise['goal'] = exercise_info[2]
            
            # add set data to main dict
            exercise_set_header = e['data'][0].split(';')
            for set in e['data'][1:]:
                complete_set = clean_exercise.copy()
                set_info = set.split(';')
                
                #clean_exercise['test'] = exercise_set_header # use to check data quality
                complete_set['set_number'] = set_info[0]

                if 'KG' in exercise_set_header:
                    complete_set['weight'] = set_info[1]
                    complete_set['quantity'] = set_info[2]
                    complete_set['measure'] = 'reps'
                elif ('KG' not in exercise_set_header and 'REPS' in exercise_set_header):
                    complete_set['weight'] = ''
                    complete_set['quantity'] = set_info[1]
                    complete_set['measure'] = 'reps'
                elif ('KG' not in exercise_set_header and 'SECS' in exercise_set_header):
                    complete_set['weight'] = ''
                    complete_set['quantity'] = set_info[1]
                    complete_set['measure'] = 'secs'
                elif ('KG' not in exercise_set_header and 'MINS' in exercise_set_header):
                    complete_set['weight'] = ''
                    complete_set['quantity'] = set_info[1]
                    complete_set['measure'] = 'mins'
                else:
                    complete_set['weight'] = 'error'
                    complete_set['quantity'] = '-1'
                    complete_set['measure'] = 'error'
                
                complete_set['comments'] = exercise_comments

                exercise_sets.append(complete_set)
                
            clean_exercises.append(exercise_sets)
        
        #workout_dict['raw_exercises'] = raw_exercises # not returned but still useful to check for data quality
        workout_dict['exercises'] = [item for sublist in clean_exercises for item in sublist]
        workout_counter += 1

        workout_listdict.append(workout_dict)
    
    
    return workout_listdict

#### Others

In [109]:
def extract_day_or_week(value):
    if value == "Standalone workout":
        return 0
    elif value.split()[-1].isdigit():
        return int(value.split()[-1])

In [110]:
def convert_to_minutes(duration):
    if "hr" in duration:
        if ':' in duration:
            hours, minutes = duration.replace(" hr", "").split(":")
            return int(hours) * 60 + int(minutes)
        else:
            return int(duration.replace(" hr", "")) * 60
    elif "min" in duration:
        return int(duration.replace(" min", ""))
    else:
        return None

In [111]:
def convert_to_seconds(duration):
    if pd.isna(duration) or duration == '' or duration == 0:
        return 0
    elif ':' in duration:
        minutes, seconds = duration.split(":")
        return int(minutes) * 60 + int(seconds)
    else:
        return int(duration) * 60

In [112]:
def strip_measure(value):
    if (pd.isna(value) or value == ''):
        return 0
    else:
        return value.split()[0]

In [113]:
def generate_exercise_id(exercise_name_column, equipment_column):
    df = pd.DataFrame()

    df['exercise_name_formatted'] = (
        exercise_name_column
        .str.lower()
        .str.replace("on ", "", regex=True)
        .str.replace(" ", "", regex=False)
        .str.replace("-", "", regex=False)
    )

    df['equipment_short'] = (
        equipment_column
        .map({
            'Bodyweight': 'BW',
            'Machine': 'MCH',
            'Cable': 'CAB',
            'Barbell': 'BB',
            'Dumbbells': 'DB',
            'Kettlebells': 'KB',
            'EZ bar': 'EZB',
            'Trap bar': 'TB',
            'TRX': 'TRX',
            'Smith machine': 'SMT',
            'Resistance bands': 'RES',
            })
        .fillna('MISSING')
    )

    return df['exercise_name_formatted'] + '_' + df['equipment_short']

### Load

In [114]:
def insert_df(df, sql_table, schema='public', chunk_size=3000):
  num_chunks = (len(df) // chunk_size) + 1

  print(f"Inserting {len(df)} rows in {num_chunks} chunks.\n")

  colunas_df = list(df.columns)

  # Connection to the PostgreSQL database
  conn = psycopg2.connect(
      host=db_host,
      database=db_name,
      user=db_user,
      password=db_password
  )
  cur = conn.cursor()

  cur.execute(f"SET search_path TO {schema}")

  for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(df))
    chunk_df = df.iloc[start_idx:end_idx]

    print(f"Inserting rows {start_idx}:{end_idx}")

    # tl == Tuple List
    tl_df = list(chunk_df.itertuples(index=False, name=None))

    insert_df = sql.SQL("INSERT INTO {} ({}) VALUES %s").format(
      sql.Identifier(sql_table),
      sql.SQL(', ').join(map(sql.Identifier, colunas_df))
    )

    # Execute the query with the data
    try:
        extras.execute_values(cur, insert_df, tl_df)
        conn.commit()
        print("Rows inserted successfuly.")
    except Exception as e:
        conn.rollback()
        print(f"Error: {e}")

  # Close the cursor and connection
  cur.close()
  conn.close()

  print("FINISHED")

  return None

# Extract

In [115]:
raw_workouts = get_raw_workouts(WORKOUTS_FILE)

# Transform

## Flattening

In [116]:
normalized_workouts_dict = get_workout_dict(raw_workouts)

In [117]:
#pprint.pprint(normalized_workouts_dict[1], sort_dicts=False) #['exercises']

## Dataframes

In [118]:
workouts_list = []
exercises_log_list = []

for i in normalized_workouts_dict:
    workouts_list.append(i['description'])
    exercises_log_list.append(i['exercises'])

exercises_log_list = [item for sublist in exercises_log_list for item in sublist]

df_workouts_orig = pd.DataFrame.from_dict(workouts_list)
df_exerciseslog_orig = pd.DataFrame.from_dict(exercises_log_list)


## Exercises Ref

### Custom Exercises

In [119]:
df_customexercises_raw = pd.read_csv(CUSTOMEXERCISES_FILE, sep=';')

In [120]:
df_customexercises_clean = (
    df_customexercises_raw
    .rename(columns={
        'NAME': 'exercise_name',
        'EQUIPMENT': 'equipment',
        'MUSCLE': 'main_muscle',
        'AUXILIARY MUSCLES': 'auxiliary_muscles',
        'TYPE': 'size',
        'METRICS': 'metrics',
        'BODYWEIGHT': 'bodyweight',
        'NOTES': 'notes',
        })
    .assign(custom_exercise = True,)
    .sort_values(by='exercise_name').reset_index(drop=True)
)

In [121]:
df_customexercises_clean = df_customexercises_clean[[
    'custom_exercise',
    'exercise_name',
    'equipment',
    'main_muscle',
    'auxiliary_muscles',
    'size',
    'metrics',
    'bodyweight',
    'notes',
]]

### Standard Exercises

In [122]:
df_all_exercises = (
    df_exerciseslog_orig[['exercise_name', 'equipment', 'workout_id']]
    .groupby(['exercise_name', 'equipment']).count().reset_index()
    .drop(columns=['workout_id'])
    #.assign(custom_exercise = False)
).merge(df_customexercises_clean[['exercise_name', 'equipment', 'custom_exercise']], how="left", on=['exercise_name', 'equipment']).fillna(False)

C:\Users\User\AppData\Local\Temp\ipykernel_19892\1032905986.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).merge(df_customexercises_clean[['exercise_name', 'equipment', 'custom_exercise']], how="left", on=['exercise_name', 'equipment']).fillna(False)


In [123]:
df_standard_exercises = (
    df_all_exercises
    .query(" custom_exercise != True ")
    .drop(columns='custom_exercise')
    .assign(
        custom_exercise = False,
        main_muscle = '',
        auxiliary_muscles = '',
        size = '',
        metrics = '',
        bodyweight = '',
        notes = ''
    ))

In [124]:
df_standard_exercises = df_standard_exercises[[
    'custom_exercise',
    'exercise_name',
    'equipment',
    'main_muscle',
    'auxiliary_muscles',
    'size',
    'metrics',
    'bodyweight',
    'notes',
]]

In [125]:
#df_standard_exercises

### Join

In [126]:
df_exercisesref_orig = pd.concat([df_customexercises_clean, df_standard_exercises]).sort_values('exercise_name', ignore_index=True)

### Grouping

In [127]:
exercise_groups = [
{'exercise_name': 'Archer Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Bilateral Calf Press - Horizontal Leg Press', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Calf Press'},
{'exercise_name': 'Chair Crunches', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Ab Crunch'},
{'exercise_name': 'Decline Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Diagonal Hip Abduction', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Abduction'},
{'exercise_name': 'Elevated Pike hold', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pike'},
{'exercise_name': 'Explosive Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Hanging Knee Tuck - iso', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Leg Raise'},
{'exercise_name': 'Incline Archer Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Incline Australian Pull-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Knee Diamond Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'L Raises', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Leg Raise'},
{'exercise_name': 'L-Sit Hold', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Leg Raise'},
{'exercise_name': 'Pike Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pike'},
{'exercise_name': 'Pseudo Planche Lean - Iso', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Pseudo Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Scapular Pull-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pull-Up'},
{'exercise_name': 'Single Leg Hip Thrust', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Thrust'},
{'exercise_name': 'Single leg deadlift', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Other'},
{'exercise_name': 'Skin the cat', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Lat Pull'},
{'exercise_name': 'Skydiver', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Back Extension'},
{'exercise_name': 'Skydiver - Iso', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Back Extension'},
{'exercise_name': 'Weighted L Raises', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Leg Raise'},
{'exercise_name': 'Assisted Chin-Ups', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Pull-Up'},
{'exercise_name': 'Assisted Dips', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Dip'},
{'exercise_name': 'Back Extensions on Roman Chair', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Back Extension'},
{'exercise_name': 'Bench Dips', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Dip'},
{'exercise_name': 'Bench Press', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Chest Press'},
{'exercise_name': 'Bench Press', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Chest Press'},
{'exercise_name': 'Bent-Over Rows', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Bulgarian Split Squats', 'equipment': 'Dumbbells', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Burpees', 'equipment': 'Bodyweight', 'muscle_group': 'Full Body', 'exercise_group': 'Cardio'},
{'exercise_name': 'Butterfly with Close Grip', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Fly'},
{'exercise_name': 'Butterfly with Wide Grip', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Fly'},
{'exercise_name': 'Calf Raises on Leg Press', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Calf Press'},
{'exercise_name': 'Chest Press', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Chest Press'},
{'exercise_name': 'Chest-Supported Low Rows with Neutral Grip', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Chest-Supported Rows with Wide Grip', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Concentration Curls', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Crunch Machine', 'equipment': 'Machine', 'muscle_group': 'Core', 'exercise_group': 'Ab Crunch'},
{'exercise_name': 'Curls', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Curls', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Curls', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Curls', 'equipment': 'EZ bar', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Decline Crunches', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Ab Crunch'},
{'exercise_name': 'Dips', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Dip'},
{'exercise_name': 'Double Crunch Machine', 'equipment': 'Machine', 'muscle_group': 'Core', 'exercise_group': 'Ab Crunch'},
{'exercise_name': 'Dragon Flag', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Other'},
{'exercise_name': 'Flys', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Fly'},
{'exercise_name': 'Flys', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Fly'},
{'exercise_name': 'Forward Lunges', 'equipment': 'Dumbbells', 'muscle_group': 'Lower Body', 'exercise_group': 'Lunge'},
{'exercise_name': 'Front Raises', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Front Raise'},
{'exercise_name': 'Front Raises', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Front Raise'},
{'exercise_name': 'Goblet Squats', 'equipment': 'Dumbbells', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Hanging Leg Raises', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Leg Raise'},
{'exercise_name': 'Hip Abduction', 'equipment': 'Cable', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Abduction'},
{'exercise_name': 'Hip Abduction', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Abduction'},
{'exercise_name': 'Hip Adduction', 'equipment': 'Cable', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Abduction'},
{'exercise_name': 'Hip Adduction', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Abduction'},
{'exercise_name': 'Hip Thrusts', 'equipment': 'Barbell', 'muscle_group': 'Lower Body', 'exercise_group': 'Hip Thrust'},
{'exercise_name': 'Incline Bench Press', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Chest Press'},
{'exercise_name': 'Incline Bench Press', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Chest Press'},
{'exercise_name': 'Incline Reverse Flys', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Reverse Fly'},
{'exercise_name': 'Incline Rows', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Inverted Rows on Smith Machine', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Inverted Rows on Smith Machine with Reverse Grip', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Kneeling Crunches', 'equipment': 'Cable', 'muscle_group': 'Core', 'exercise_group': 'Ab Crunch'},
{'exercise_name': 'Lat Pulldowns with Close Neutral Grip', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Pulldown'},
{'exercise_name': 'Lat Pulldowns with Wide Neutral Grip', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Pulldown'},
{'exercise_name': 'Lat Pulldowns with Wide Overhand Grip', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Pulldown'},
{'exercise_name': 'Lateral Raises', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Lateral Raise'},
{'exercise_name': 'Lateral Raises', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Lateral Raise'},
{'exercise_name': 'Lean Away Lateral Raises', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Lateral Raise'},
{'exercise_name': 'Leg Curls on Leg Extension Machine', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Curl'},
{'exercise_name': 'Leg Extensions', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Extension'},
{'exercise_name': 'Leg Press', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Press'},
{'exercise_name': 'Leg Raises', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Leg Raise'},
{'exercise_name': 'Lying Leg Curls', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Curl'},
{'exercise_name': 'One-Arm Rows', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Overhead Triceps Extensions', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Triceps Extension'},
{'exercise_name': 'Pistol Squats', 'equipment': 'TRX', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Plank', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Plank'},
{'exercise_name': 'Preacher Curls', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Pull-Ups with Close Neutral Grip', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pull-Up'},
{'exercise_name': 'Pull-Ups with Wide Overhand Grip', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pull-Up'},
{'exercise_name': 'Pullovers', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Lat Pull'},
{'exercise_name': 'Push-Ups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Reverse Butterfly with Wide Grip', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Reverse Fly'},
{'exercise_name': 'Romanian Deadlifts', 'equipment': 'Barbell', 'muscle_group': 'Lower Body', 'exercise_group': 'Deadlift'},
{'exercise_name': 'Rows with Close Grip', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Rows with Close Grip', 'equipment': 'TRX', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Rows with Reverse Grip', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Rows with Wide Neutral Grip', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Russian Swings', 'equipment': 'Kettlebells', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Russian Twists', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Other'},
{'exercise_name': 'Seated Calf Raises', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Calf Press'},
{'exercise_name': 'Seated Curls', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Seated Lateral Raises', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Lateral Raise'},
{'exercise_name': 'Seated Leg Curls', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Curl'},
{'exercise_name': 'Seated Shoulder Press', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Shoulder Press'},
{'exercise_name': 'Seated Shoulder Press', 'equipment': 'Dumbbells', 'muscle_group': 'Upper Body', 'exercise_group': 'Shoulder Press'},
{'exercise_name': 'Shoulder Press', 'equipment': 'Machine', 'muscle_group': 'Upper Body', 'exercise_group': 'Shoulder Press'},
{'exercise_name': 'Side Bends on Roman Chair', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Side Abs'},
{'exercise_name': 'Side Plank', 'equipment': 'Bodyweight', 'muscle_group': 'Core', 'exercise_group': 'Side Abs'},
{'exercise_name': 'Side Split Squats', 'equipment': 'Dumbbells', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Skull Crushers', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Triceps Extension'},
{'exercise_name': 'Skull Crushers', 'equipment': 'EZ bar', 'muscle_group': 'Upper Body', 'exercise_group': 'Triceps Extension'},
{'exercise_name': 'Squats', 'equipment': 'Barbell', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Squats', 'equipment': 'Smith machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Standing Calf Raises', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Calf Press'},
{'exercise_name': 'Standing Calf Raises', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Calf Press'},
{'exercise_name': 'Standing Shoulder Press', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Shoulder Press'},
{'exercise_name': 'Triceps Pushdowns', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Triceps Extension'},
{'exercise_name': 'Triceps Pushdowns with Rope', 'equipment': 'Cable', 'muscle_group': 'Upper Body', 'exercise_group': 'Triceps Extension'},
{'exercise_name': 'Upright Rows', 'equipment': 'Barbell', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Elevated Dragon Squats', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Frog stand hold', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Front Raise'},
{'exercise_name': 'Full Nordic Curls', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Curl'},
{'exercise_name': 'Natural Leg Extensions', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Extension'},
{'exercise_name': 'Negative Nordic Curls', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Leg Curl'},
{'exercise_name': 'Shrimp Squats', 'equipment': 'Bodyweight', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Sumo Belt Squats', 'equipment': 'Machine', 'muscle_group': 'Lower Body', 'exercise_group': 'Squat'},
{'exercise_name': 'Typewriter pullups', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pull-Up'},
{'exercise_name': 'Wall Handstand Hold', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pike'},
{'exercise_name': 'Chest Press', 'equipment': 'TRX', 'muscle_group': 'Upper Body', 'exercise_group': 'Chest Press'},
{'exercise_name': 'Curls with Raised Arms', 'equipment': 'TRX', 'muscle_group': 'Upper Body', 'exercise_group': 'Biceps Curl'},
{'exercise_name': 'Flys', 'equipment': 'TRX', 'muscle_group': 'Upper Body', 'exercise_group': 'Fly'},
{'exercise_name': 'One-Arm Rows', 'equipment': 'TRX', 'muscle_group': 'Upper Body', 'exercise_group': 'Row'},
{'exercise_name': 'Pull-Ups with Wide Neutral Grip', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Pull-Up'},
{'exercise_name': 'Push-Ups with Wide Grip', 'equipment': 'Bodyweight', 'muscle_group': 'Upper Body', 'exercise_group': 'Push-Up'},
{'exercise_name': 'Reverse Flys', 'equipment': 'TRX', 'muscle_group': 'Upper Body', 'exercise_group': 'Reverse Fly'},
{'exercise_name': '', 'equipment': '', 'muscle_group': '', 'exercise_group': ''},
]

df_exercise_grouping = pd.DataFrame.from_dict(exercise_groups)

In [128]:
df_exercisesref_groups = df_exercisesref_orig.merge(df_exercise_grouping, how='left', on=['exercise_name', 'equipment'])
df_exercisesref_groups['muscle_group'] = df_exercisesref_groups['muscle_group'].fillna('MISSING')

### Normalization

In [129]:
# Decided to skip this for now

In [130]:
df_exercisesref_normalized = df_exercisesref_groups.copy()

In [131]:
#df_exercisesref_normalized["auxiliary_muscles"] = df_exercisesref_groups["auxiliary_muscles"].str.split(", ")

In [132]:
hold_aux_muscles = '''
all_auxiliary_muscles = []
for muscles in df_exercisesref_normalized["auxiliary_muscles"]:
    for muscle in muscles:
        if muscle not in all_auxiliary_muscles:
            all_auxiliary_muscles.append(f"aux_{muscle.lower()}")

all_auxiliary_muscles.sort()
all_auxiliary_muscles.pop(0)
all_auxiliary_muscles.pop(0)

for muscle in all_auxiliary_muscles:
    df_exercisesref_normalized[muscle] = df_exercisesref_normalized["auxiliary_muscles"].apply(lambda x: muscle in x)
'''

### Final

In [133]:
df_exercisesref_final = df_exercisesref_normalized.copy()

df_exercisesref_final['id'] = generate_exercise_id(df_exercisesref_normalized['exercise_name'], df_exercisesref_normalized['equipment'])

In [134]:
#df_exercisesref_final[['exercise_name', 'equipment', 'exercise_name_formatted', 'equipment_short', 'id']]#.query(" equipment_short == 'MISSING' ")

In [135]:
df_exercisesref_final = df_exercisesref_final[[
    'id',
    'custom_exercise',
    'exercise_name',
    'equipment',
    'muscle_group',
    'exercise_group',
    'main_muscle',
    'auxiliary_muscles',
    'size',
    'metrics',
    'bodyweight',
    'notes'
]]

In [136]:
#df_exercisesref_final

## Workouts

In [137]:
df_workouts_formatted = df_workouts_orig.copy()

In [138]:
df_workouts_formatted['day_number'] = df_workouts_orig['day_number'].apply(extract_day_or_week).astype('string')
df_workouts_formatted['week_number'] = df_workouts_orig['week_number'].apply(extract_day_or_week).astype('string')
df_workouts_formatted['start_time'] = pd.to_datetime(df_workouts_orig['start_time'], format="%Y-%m-%d %H:%M h")
df_workouts_formatted['duration'] = df_workouts_orig['duration'].apply(convert_to_minutes)
df_workouts_formatted['end_time'] = df_workouts_formatted['start_time'] + pd.to_timedelta(df_workouts_formatted['duration'], unit='m')

In [139]:
#df_workouts_formatted.iloc[:20]

## Exercises Log

In [140]:
df_exerciseslog_formatted = df_exerciseslog_orig.copy()

In [143]:
df_exerciseslog_formatted['id'] =  df_exerciseslog_formatted['workout_id'] + '_' + df_exerciseslog_formatted['exercise_number'] + '_' + df_exerciseslog_formatted['set_number']
df_exerciseslog_formatted['exercise_id'] = generate_exercise_id(df_exerciseslog_formatted['exercise_name'], df_exerciseslog_formatted['equipment'])

In [145]:
# convert values in minutes to seconds
mask_mins = (df_exerciseslog_formatted["measure"] == "mins")

df_exerciseslog_formatted['goal'] = df_exerciseslog_formatted['goal'].apply(strip_measure)
df_exerciseslog_formatted.loc[mask_mins, "goal"] = df_exerciseslog_formatted.loc[mask_mins]['goal'].apply(convert_to_seconds)
df_exerciseslog_formatted.loc[mask_mins, "quantity"] = df_exerciseslog_orig.loc[mask_mins, "quantity"].apply(convert_to_seconds)
df_exerciseslog_formatted.loc[df_exerciseslog_orig["measure"] == "mins", "measure"] = "secs"

In [146]:
df_exerciseslog_formatted['goal'] = df_exerciseslog_formatted['goal'].fillna(0).astype('int64')
df_exerciseslog_formatted['quantity'] = df_exerciseslog_formatted['quantity'].replace('-', 0).astype('int64')
df_exerciseslog_formatted['weight'] = df_exerciseslog_formatted['weight'].str.replace("+", "").str.replace(",", ".").replace('-', 0).replace('', 0).fillna(0).astype('float64')

In [147]:
df_exerciseslog_formatted['volume'] = df_exerciseslog_formatted['weight'] * df_exerciseslog_formatted['quantity']

In [148]:
# when an exercise uses dumbbells, multiply its weight by 2 to yield total weight

df_exerciseslog_formatted['total_weight'] = df_exerciseslog_formatted['weight']

mask_dumbbells = (df_exerciseslog_formatted["equipment"] == "Dumbbells")
df_exerciseslog_formatted.loc[mask_dumbbells, "total_weight"] = df_exerciseslog_formatted.loc[mask_dumbbells, "total_weight"] * 2

In [150]:
df_exerciseslog_formatted = df_exerciseslog_formatted[[
    'id',
    'exercise_id',
    'workout_id',
    'exercise_number',
    'exercise_name',
    'equipment',
    'goal',
    'measure',
    'set_number',
    'weight',
    'total_weight',
    'quantity',
    'volume'
]]

# Load

In [156]:
#df_workouts_formatted.to_csv(f"{GOLD_FOLDER}/workouts.csv", sep=";", decimal=',', index=False)
#df_exercisesref_final.to_csv(f"{GOLD_FOLDER}/exercises_ref.csv", sep=";", decimal=',', index=False)
#df_exerciseslog_formatted.to_csv(f"{GOLD_FOLDER}/exercises_log.csv", sep=";", decimal=',', index=False)

In [162]:
#insert_df(df_workouts_formatted, 'gold_dim_workouts', 'ap_workouts')

In [ ]:
#insert_df(df_exercisesref_final, 'gold_dim_exercisesref', 'ap_workouts')

In [ ]:
#insert_df(df_exerciseslog_formatted, 'gold_fact_sets', 'ap_workouts')

# Tests

In [70]:
#df_workouts_formatted

In [87]:
#df_exerciseslog_formatted

In [88]:
#df_exercises_ref